In [1]:
! pip install yfinance langchain_pinecone openai python-dotenv langchain-community sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
from langchain_pinecone import PineconeVectorStore
from openai import OpenAI
import dotenv
import json
import yfinance as yf
import concurrent.futures
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from langchain.schema import Document
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import numpy as np
import requests
import os

In [3]:
def get_all_stock_properties(symbol: str):
    """
    Retrieves and lists all available stock properties from Yahoo Finance.

    Args:
        symbol (str): The stock ticker symbol to look up.

    Returns:
        list: A list of all available properties.
    """
    try:
        data = yf.Ticker(symbol)
        stock_info = data.info  # Retrieves all available information

        # Print all keys (property names)
        properties = list(stock_info.keys())
        print("\n".join(properties))

        return properties  # Returns the full list of properties

    except Exception as e:
        print(f"Error retrieving stock properties: {e}")
        return []

# Example usage:
get_all_stock_properties("AAPL")  # Replace with any stock ticker


address1
city
state
zip
country
phone
website
industry
industryKey
industryDisp
sector
sectorKey
sectorDisp
longBusinessSummary
fullTimeEmployees
companyOfficers
auditRisk
boardRisk
compensationRisk
shareHolderRightsRisk
overallRisk
governanceEpochDate
compensationAsOfEpochDate
irWebsite
executiveTeam
maxAge
priceHint
previousClose
open
dayLow
dayHigh
regularMarketPreviousClose
regularMarketOpen
regularMarketDayLow
regularMarketDayHigh
dividendRate
dividendYield
exDividendDate
payoutRatio
fiveYearAvgDividendYield
beta
trailingPE
forwardPE
volume
regularMarketVolume
averageVolume
averageVolume10days
averageDailyVolume10Day
bid
ask
bidSize
askSize
marketCap
fiftyTwoWeekLow
fiftyTwoWeekHigh
priceToSalesTrailing12Months
fiftyDayAverage
twoHundredDayAverage
trailingAnnualDividendRate
trailingAnnualDividendYield
currency
tradeable
enterpriseValue
profitMargins
floatShares
sharesOutstanding
sharesShort
sharesShortPriorMonth
sharesShortPreviousMonthDate
dateShortInterest
sharesPercentSharesOut

['address1',
 'city',
 'state',
 'zip',
 'country',
 'phone',
 'website',
 'industry',
 'industryKey',
 'industryDisp',
 'sector',
 'sectorKey',
 'sectorDisp',
 'longBusinessSummary',
 'fullTimeEmployees',
 'companyOfficers',
 'auditRisk',
 'boardRisk',
 'compensationRisk',
 'shareHolderRightsRisk',
 'overallRisk',
 'governanceEpochDate',
 'compensationAsOfEpochDate',
 'irWebsite',
 'executiveTeam',
 'maxAge',
 'priceHint',
 'previousClose',
 'open',
 'dayLow',
 'dayHigh',
 'regularMarketPreviousClose',
 'regularMarketOpen',
 'regularMarketDayLow',
 'regularMarketDayHigh',
 'dividendRate',
 'dividendYield',
 'exDividendDate',
 'payoutRatio',
 'fiveYearAvgDividendYield',
 'beta',
 'trailingPE',
 'forwardPE',
 'volume',
 'regularMarketVolume',
 'averageVolume',
 'averageVolume10days',
 'averageDailyVolume10Day',
 'bid',
 'ask',
 'bidSize',
 'askSize',
 'marketCap',
 'fiftyTwoWeekLow',
 'fiftyTwoWeekHigh',
 'priceToSalesTrailing12Months',
 'fiftyDayAverage',
 'twoHundredDayAverage',
 'tra

In [7]:
def get_stock_info(symbol: str) -> dict:
    """
    Retrieves and formats detailed information about a stock from Yahoo Finance.

    Args:
        symbol (str): The stock ticker symbol to look up.

    Returns:
        dict: A dictionary containing detailed stock information, including ticker, name,
              business summary, city, state, country, industry, and sector.
    """
    data = yf.Ticker(symbol)
    stock_info = data.info

    properties = {
        "Ticker": stock_info.get('symbol', 'Information not available'),
        'Name': stock_info.get('longName', 'Information not available'),
        'Business Summary': stock_info.get('longBusinessSummary'),
        'Sector': stock_info.get('sector', 'Information not available'),
        'Industry': stock_info.get('industry', 'Information not available'),
        'City': stock_info.get('city', 'Information not available'),
        'State': stock_info.get('state', 'Information not available'),
        'Country': stock_info.get('country', 'Information not available'),
        'Website': stock_info.get('website', 'Information not available'),



        # Stock Prices
        "Current Price": stock_info.get('currentPrice', 'Information not available'),
        "Previous Close": stock_info.get('previousClose', 'Information not available'),
        "52-Week High": stock_info.get('fiftyTwoWeekHigh', 'Information not available'),
        "52-Week Low": stock_info.get('fiftyTwoWeekLow', 'Information not available'),
        "Market Cap": stock_info.get('marketCap', 'Information not available'),
        "Volume": stock_info.get('volume', 'Information not available'),
        "Average Volume (10-day)": stock_info.get('averageDailyVolume10Day', 'Information not available'),

        # Valuation & Ratios
        "P/E Ratio": stock_info.get('trailingPE', 'Information not available'),
        "Forward P/E": stock_info.get('forwardPE', 'Information not available'),
        "Price to Book": stock_info.get('priceToBook', 'Information not available'),
        "Profit Margins": stock_info.get('profitMargins', 'Information not available'),

        # Growth & Financial Health
        "Revenue Growth": stock_info.get('revenueGrowth', 'Information not available'),
        "Earnings Growth": stock_info.get('earningsGrowth', 'Information not available'),
        "Return on Assets": stock_info.get('returnOnAssets', 'Information not available'),
        "Return on Equity": stock_info.get('returnOnEquity', 'Information not available'),
        "Debt to Equity": stock_info.get('debtToEquity', 'Information not available'),

        # Dividend Info
        "Dividend Yield": stock_info.get('dividendYield', 'Information not available'),
        "Dividend Rate": stock_info.get('dividendRate', 'Information not available'),

        # Analyst Ratings
        "Analyst Rating": stock_info.get('recommendationKey', 'Information not available'),
        "Target High Price": stock_info.get('targetHighPrice', 'Information not available'),
        "Target Low Price": stock_info.get('targetLowPrice', 'Information not available'),



    }

    return properties

In [5]:
aapl_info = get_stock_info('AAPL')
print(aapl_info)

{'Ticker': 'AAPL', 'Name': 'Apple Inc.', 'Business Summary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as well as advertising services include third-party licensing arrangements and its own advertising platforms. In addition, the company offers various subscription-based services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening experien

In [9]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    """
    Generates embeddings for the given text using a specified Hugging Face model.

    Args:
        text (str): The input text to generate embeddings for.
        model_name (str): The name of the Hugging Face model to use.
                          Defaults to "sentence-transformers/all-mpnet-base-v2".

    Returns:
        np.ndarray: The generated embeddings as a NumPy array.
    """
    model = SentenceTransformer(model_name)
    return model.encode(text)


def cosine_similarity_between_sentences(sentence1, sentence2):
    """
    Calculates the cosine similarity between two sentences.

    Args:
        sentence1 (str): The first sentence for similarity comparison.
        sentence2 (str): The second sentence for similarity comparison.

    Returns:
        float: The cosine similarity score between the two sentences,
               ranging from -1 (completely opposite) to 1 (identical).

    Notes:
        Prints the similarity score to the console in a formatted string.
    """
    # Get embeddings for both sentences
    embedding1 = np.array(get_huggingface_embeddings(sentence1))
    embedding2 = np.array(get_huggingface_embeddings(sentence2))

    # Reshape embeddings for cosine_similarity function
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    similarity_score = similarity[0][0]
    print(f"Cosine similarity between the two sentences: {similarity_score:.4f}")
    return similarity_score


# Example usage
sentence1 = "I like walking to the park"
sentence2 = "I like running to the office"

similarity = cosine_similarity_between_sentences(sentence1, sentence2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Cosine similarity between the two sentences: 0.6133


In [10]:
aapl_description = aapl_info['Business Summary']

company_description = "I want to find companies that make smartphones and are headquarted in California"

similarity = cosine_similarity_between_sentences(aapl_description, company_description)

Cosine similarity between the two sentences: 0.3635


In [11]:
def get_company_tickers():
    """
    Downloads and parses the Stock ticker symbols from the GitHub-hosted SEC company tickers JSON file.

    Returns:
        dict: A dictionary containing company tickers and related information.

    Notes:
        The data is sourced from the official SEC website via a GitHub repository:
        https://raw.githubusercontent.com/team-headstart/Financial-Analysis-and-Automation-with-LLMs/main/company_tickers.json
    """
    # URL to fetch the raw JSON file from GitHub
    url = "https://raw.githubusercontent.com/team-headstart/Financial-Analysis-and-Automation-with-LLMs/main/company_tickers.json"

    # Making a GET request to the URL
    response = requests.get(url)

    # Checking if the request was successful
    if response.status_code == 200:
        # Parse the JSON content directly
        company_tickers = json.loads(response.content.decode('utf-8'))

        # Optionally save the content to a local file for future use
        with open("company_tickers.json", "w", encoding="utf-8") as file:
            json.dump(company_tickers, file, indent=4)

        print("File downloaded successfully and saved as 'company_tickers.json'")
        return company_tickers
    else:
        print(f"Failed to download file. Status code: {response.status_code}")
        return None

company_tickers = get_company_tickers()

File downloaded successfully and saved as 'company_tickers.json'


In [12]:
company_tickers

{'0': {'cik_str': 1045810, 'ticker': 'NVDA', 'title': 'NVIDIA CORP'},
 '1': {'cik_str': 320193, 'ticker': 'AAPL', 'title': 'Apple Inc.'},
 '2': {'cik_str': 789019, 'ticker': 'MSFT', 'title': 'MICROSOFT CORP'},
 '3': {'cik_str': 1018724, 'ticker': 'AMZN', 'title': 'AMAZON COM INC'},
 '4': {'cik_str': 1652044, 'ticker': 'GOOGL', 'title': 'Alphabet Inc.'},
 '5': {'cik_str': 1326801, 'ticker': 'META', 'title': 'Meta Platforms, Inc.'},
 '6': {'cik_str': 1318605, 'ticker': 'TSLA', 'title': 'Tesla, Inc.'},
 '7': {'cik_str': 1067983,
  'ticker': 'BRK-B',
  'title': 'BERKSHIRE HATHAWAY INC'},
 '8': {'cik_str': 1046179,
  'ticker': 'TSM',
  'title': 'TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD'},
 '9': {'cik_str': 1730168, 'ticker': 'AVGO', 'title': 'Broadcom Inc.'},
 '10': {'cik_str': 59478, 'ticker': 'LLY', 'title': 'ELI LILLY & Co'},
 '11': {'cik_str': 19617, 'ticker': 'JPM', 'title': 'JPMORGAN CHASE & CO'},
 '12': {'cik_str': 104169, 'ticker': 'WMT', 'title': 'Walmart Inc.'},
 '13': {'cik_str'

In [13]:
len(company_tickers)

9998

In [14]:
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

index_name = "stocks"
namespace = "stock-descriptions"

hf_embeddings = HuggingFaceEmbeddings()
vectorstore = PineconeVectorStore(index_name=index_name, embedding=hf_embeddings)

<ipython-input-14-8d5c0f356cf6>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings()
<ipython-input-14-8d5c0f356cf6>:7: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hf_embeddings = HuggingFaceEmbeddings()


In [15]:
for idx, stock in company_tickers.items():
    stock_ticker = stock['ticker']
    stock_data = get_stock_info(stock_ticker)
    stock_description = stock_data['Business Summary']

    print(f"Processing stock {idx} / {len(company_tickers)} :", stock_ticker)

    vectorstore_from_documents = PineconeVectorStore.from_documents(
        documents=[Document(page_content=stock_description, metadata=stock_data)],
        embedding=hf_embeddings,
        index_name=index_name,
        namespace=namespace
    )

Processing stock 0 / 9998 : NVDA
Processing stock 1 / 9998 : AAPL
Processing stock 2 / 9998 : MSFT
Processing stock 3 / 9998 : AMZN
Processing stock 4 / 9998 : GOOGL
Processing stock 5 / 9998 : META
Processing stock 6 / 9998 : TSLA
Processing stock 7 / 9998 : BRK-B
Processing stock 8 / 9998 : TSM
Processing stock 9 / 9998 : AVGO
Processing stock 10 / 9998 : LLY
Processing stock 11 / 9998 : JPM
Processing stock 12 / 9998 : WMT
Processing stock 13 / 9998 : V
Processing stock 14 / 9998 : UNH
Processing stock 15 / 9998 : SPY
Processing stock 16 / 9998 : XOM
Processing stock 17 / 9998 : ORCL
Processing stock 18 / 9998 : MA
Processing stock 19 / 9998 : NVO
Processing stock 20 / 9998 : HD
Processing stock 21 / 9998 : COST
Processing stock 22 / 9998 : PG
Processing stock 23 / 9998 : JNJ
Processing stock 24 / 9998 : BAC
Processing stock 25 / 9998 : NFLX
Processing stock 26 / 9998 : CRM
Processing stock 27 / 9998 : ABBV
Processing stock 28 / 9998 : RCIT
Processing stock 29 / 9998 : CVX
Processin

IndexError: list index out of range

In [16]:
# Initialize tracking lists
successful_tickers = []
unsuccessful_tickers = []

# Load existing successful/unsuccessful tickers
try:
    with open('successful_tickers.txt', 'r') as f:
        successful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(successful_tickers)} successful tickers")
except FileNotFoundError:
    print("No existing successful tickers file found")

try:
    with open('unsuccessful_tickers.txt', 'r') as f:
        unsuccessful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(unsuccessful_tickers)} unsuccessful tickers")
except FileNotFoundError:
    print("No existing unsuccessful tickers file found")

No existing successful tickers file found
No existing unsuccessful tickers file found


In [17]:
def process_stock(stock_ticker: str) -> str:
    # Skip if already processed
    if stock_ticker in successful_tickers:
        return f"Already processed {stock_ticker}"

    try:
        # Get and store stock data
        stock_data = get_stock_info(stock_ticker)
        stock_description = stock_data['Business Summary']

        # Store stock description in Pinecone
        vectorstore_from_texts = PineconeVectorStore.from_documents(
            documents=[Document(page_content=stock_description, metadata=stock_data)],
            embedding=hf_embeddings,
            index_name=index_name,
            namespace=namespace
        )

        # Track success
        with open('successful_tickers.txt', 'a') as f:
            f.write(f"{stock_ticker}\n")
        successful_tickers.append(stock_ticker)

        return f"Processed {stock_ticker} successfully"

    except Exception as e:
        # Track failure
        with open('unsuccessful_tickers.txt', 'a') as f:
            f.write(f"{stock_ticker}\n")
        unsuccessful_tickers.append(stock_ticker)

        return f"ERROR processing {stock_ticker}: {e}"

def parallel_process_stocks(tickers: list, max_workers: int = 10) -> None:
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_ticker = {
            executor.submit(process_stock, ticker): ticker
            for ticker in tickers
        }

        for future in concurrent.futures.as_completed(future_to_ticker):
            ticker = future_to_ticker[future]
            try:
                result = future.result()
                print(result)

                # Stop on error
                if result.startswith("ERROR"):
                    print(f"Stopping program due to error in {ticker}")
                    executor.shutdown(wait=False)
                    raise SystemExit(1)

            except Exception as exc:
                print(f'{ticker} generated an exception: {exc}')
                print("Stopping program due to exception")
                executor.shutdown(wait=False)
                raise SystemExit(1)

# Prepare your tickers
tickers_to_process = [company_tickers[num]['ticker'] for num in company_tickers.keys()]

# Process them
parallel_process_stocks(tickers_to_process, max_workers=10)

Processed MSFT successfully
Processed TSM successfully
Processed TSLA successfully
Processed AVGO successfully
Processed AMZN successfully
Processed NVDA successfully
Processed BRK-B successfully
Processed META successfully
Processed GOOGL successfully
Processed AAPL successfully
Processed V successfully
Processed SPY successfully
Processed NVO successfully
Processed MA successfully
Processed XOM successfully
Processed UNH successfully
Processed JPM successfully
Processed LLY successfully
Processed WMT successfully
Processed ORCL successfully
Processed HD successfully
Processed JNJ successfully
Processed NFLX successfully
Processed PG successfully
Processed COST successfully
Processed CVX successfully
Processed ABBV successfully
Processed BAC successfully
Processed CRM successfully
Processed RCIT successfully
Processed TMUS successfully
Processed SAP successfully
Processed KO successfully
Processed MRK successfully
Processed TM successfully
Processed ADBE successfully
Processed CSCO su

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BGNE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BGNE&crumb=zPRpGxaBpOS
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AGR?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AGR&crumb=zPRpGxaBpOS
ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CTLT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CTLT&crumb=zPRpGxaBpOS


SystemExit: 1

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [18]:
# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index(index_name)

In [19]:
query = "What are some companies that manufacture consumer hardware?"

In [20]:
raw_query_embedding = get_huggingface_embeddings(query)

In [21]:
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=10, include_metadata=True, namespace=namespace)

In [22]:
top_matches

{'matches': [{'id': '66e204aa-abbf-4b8b-b5e1-f413d7af5135',
              'metadata': {'52-Week High': 156.35,
                           '52-Week Low': 81.229095,
                           'Analyst Rating': 'buy',
                           'Average Volume (10-day)': 5675910.0,
                           'Business Summary': '3M Company provides '
                                               'diversified technology '
                                               'services in the Americas, the '
                                               'Asia Pacific, Europe, the '
                                               'Middle East, Africa, and '
                                               'internationally. It operates '
                                               'through three segments: Safety '
                                               'and Industrial, Transportation '
                                               'and Electronics, and Consumer. '
                       

In [23]:
contexts = [item['metadata']['text'] for item in top_matches['matches']]

In [24]:
contexts

['3M Company provides diversified technology services in the Americas, the Asia Pacific, Europe, the Middle East, Africa, and internationally. It operates through three segments: Safety and Industrial, Transportation and Electronics, and Consumer. The Safety and Industrial segment offers industrial abrasives and finishing for metalworking applications; autobody repair solutions; industrial specialty products, such as personal hygiene products, masking, and packaging materials; electrical products and materials for construction and maintenance, power distribution, and electrical original equipment manufacturers; structural adhesives and tapes; respiratory, hearing, eye, and fall protection solutions; and natural and color-coated mineral granules for shingles. The Transportation and Electronics segment provides ceramic solutions; attachment/bonding, films, sound, and temperature management for transportation vehicles; format graphic films for advertising and fleet signage; reflective sig

In [25]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[:10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [26]:
print(augmented_query)

<CONTEXT>
3M Company provides diversified technology services in the Americas, the Asia Pacific, Europe, the Middle East, Africa, and internationally. It operates through three segments: Safety and Industrial, Transportation and Electronics, and Consumer. The Safety and Industrial segment offers industrial abrasives and finishing for metalworking applications; autobody repair solutions; industrial specialty products, such as personal hygiene products, masking, and packaging materials; electrical products and materials for construction and maintenance, power distribution, and electrical original equipment manufacturers; structural adhesives and tapes; respiratory, hearing, eye, and fall protection solutions; and natural and color-coated mineral granules for shingles. The Transportation and Electronics segment provides ceramic solutions; attachment/bonding, films, sound, and temperature management for transportation vehicles; format graphic films for advertising and fleet signage; reflec

In [53]:
import os
os.environ["GROQ_API_KEY"] = "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"


In [54]:
import os
print("GROQ_API_KEY:", os.getenv("GROQ_API_KEY"))


GROQ_API_KEY: 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


In [55]:
client = OpenAI(
  base_url="https://api.groq.com/openai/v1",
  api_key=os.getenv("GROQ_API_KEY")
)

In [58]:
print(client.models.list())


SyncPage[Model](data=[Model(id='llama-3.2-1b-preview', created=1727224268, object='model', owned_by='Meta', active=True, context_window=8192, public_apps=None), Model(id='distil-whisper-large-v3-en', created=1693721698, object='model', owned_by='Hugging Face', active=True, context_window=448, public_apps=None), Model(id='llama-guard-3-8b', created=1693721698, object='model', owned_by='Meta', active=True, context_window=8192, public_apps=None), Model(id='llama-3.1-8b-instant', created=1693721698, object='model', owned_by='Meta', active=True, context_window=131072, public_apps=None), Model(id='llama-3.3-70b-specdec', created=1733505017, object='model', owned_by='Meta', active=True, context_window=8192, public_apps=None), Model(id='allam-2-7b', created=1737672203, object='model', owned_by='SDAIA', active=True, context_window=4096, public_apps=None), Model(id='gemma2-9b-it', created=1693721698, object='model', owned_by='Google', active=True, context_window=8192, public_apps=None), Model(id

In [59]:
system_prompt = f"""You are a financial expert at providing answers about stocks. Please answer my question provided.
            Analyze the stocks' in detail and explain current performance and potential future trends.
            Identify any notable connections or relationships with other stocks (e.g., industry, market correlation, or shared factors).
            Provide a concise, actionable insight to guide investment decisions.

"""

llm_response = client.chat.completions.create(
    model="llama-3.2-1b-preview",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]

)

response = llm_response.choices[0].message.content

In [60]:
print(response)

Based on the provided context, here are some companies that manufacture consumer hardware:

1. **Home Depot**: An American retail giant that operates a wide range of hardware products for home improvement, including power tools, plumbing fixtures, and building materials.

2. **Lowe's**: Another well-known home improvement retailer that offers a variety of consumer hardware products, including plumbing, electrical, and construction materials.

3. **Walmart**: The world's largest retailer, offering a vast range of consumer hardware products, including home and garden tools, appliances, and accessories.

4. **Best Buy**: While primarily known for electronics, Best Buy also operates a physical retail store for consumer hardware products, such as home appliances, tools, and electronics.

5. **Ace Hardware**: A hardware retail chain that provides a wide range of consumer hardware products, including tools, plumbing and electrical supplies, and materials.

6. ** Northern Tool + Equipment**: A

In [75]:
pip install gradio

In [74]:
import gradio as gr
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load API key from .env file
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Initialize Groq API client
client = OpenAI(api_key=GROQ_API_KEY, base_url="https://api.groq.com/openai/v1")

# Define system prompt
system_prompt = """You are a financial expert at providing answers about stocks.
Analyze stock performance, future trends, industry relationships, and provide investment insights."""

# Function to interact with the Groq API
def get_stock_analysis(query):
    try:
        llm_response = client.chat.completions.create(
            model="llama-3.2-1b-preview",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": query}
            ]
        )

        return llm_response.choices[0].message.content

    except Exception as e:
        return f"Error: {str(e)}"

# Create Gradio UI
iface = gr.Interface(
    fn=get_stock_analysis,
    inputs=gr.Textbox(label="Enter Stock Query"),
    outputs=gr.Textbox(label="Stock Analysis"),
    title="AI Stock Market Analyst",
    description="Ask about stocks and get AI-powered insights on trends and investment strategies."
)

# Launch Gradio app
iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://11c0495d8a7c802aef.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
